In [0]:
client_id     = dbutils.secrets.get(scope=scope_name, key="secret-client-id")
tenant_id     = dbutils.secrets.get(scope=scope_name, key="secret-tenant-id")
client_secret = dbutils.secrets.get(scope=scope_name, key="secret-client-mdp")

storage_account = "oliststorageaccountbth74"
container_name = "data"

spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

files=["olist_customers_dataset", "olist_geolocation_dataset", "olist_order_items_dataset", "olist_order_payments_dataset", "olist_order_reviews_dataset", "olist_orders_dataset", "olist_products_dataset", "olist_sellers_dataset", "product_category_name_translation"]
df={}
for file in files:
  df[file] = spark.read.csv(f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/bronze/{file}.csv", header=True, inferSchema=True)

  silver_path=f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/silver/{file}/"
  df[file].write.format("delta").mode("overwrite").save(silver_path)
